[![Fixel Algorithms](https://fixelalgorithms.co/images/CCExt.png)](https://fixelalgorithms.gitlab.io)

# Optimization Methods

## Convex Optimization - Algorithms & Solvers - Alternating Direction Method of Multipliers (ADMM)

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 1.0.000 | 04/10/2024 | Royi Avital | First version                                                      |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/AIProgram/2024_02/0012LinearFitL1.ipynb)

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

# Machine Learning

# Optimization
import cvxpy as cp
from sksparse.cholmod import cholesky

# Miscellaneous
import math
from platform import python_version
import random
import time

# Typing
from typing import Callable, List, Tuple, Union

# Visualization
import matplotlib.pyplot as plt

# Jupyter
from IPython import get_ipython

## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

Code Notations:

```python
someVar    = 2; #<! Notation for a variable
vVector    = np.random.rand(4) #<! Notation for 1D array
mMatrix    = np.random.rand(4, 3) #<! Notation for 2D array
tTensor    = np.random.rand(4, 3, 2, 3) #<! Notation for nD array (Tensor)
tuTuple    = (1, 2, 3) #<! Notation for a tuple
lList      = [1, 2, 3] #<! Notation for a list
dDict      = {1: 3, 2: 2, 3: 1} #<! Notation for a dictionary
oObj       = MyClass() #<! Notation for an object
dfData     = pd.DataFrame() #<! Notation for a data frame
dsData     = pd.Series() #<! Notation for a series
hObj       = plt.Axes() #<! Notation for an object / handler / function handler
```

### Code Exercise

 - Single line fill

 ```python
 vallToFill = ???
 ```

 - Multi Line to Fill (At least one)

 ```python
 # You need to start writing
 ????
 ```

 - Section to Fill

```python
#===========================Fill This===========================#
# 1. Explanation about what to do.
# !! Remarks to follow / take under consideration.
mX = ???

???
#===============================================================#
```

In [ ]:
# Configuration
# %matplotlib inline

# warnings.filterwarnings("ignore")

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# Matplotlib default color palette
lMatPltLibclr = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
# sns.set_theme() #>! Apply SeaBorn theme
# sns.set_palette("tab10")

runInGoogleColab = 'google.colab' in str(get_ipython())

In [ ]:
# Constants

FIG_SIZE_DEF    = (8, 8)
ELM_SIZE_DEF    = 50
CLASS_COLOR     = ('b', 'r')
EDGE_COLOR      = 'k'
MARKER_SIZE_DEF = 10
LINE_WIDTH_DEF  = 2


In [ ]:
# Course Packages

from AuxFun import ProxGradientDescent
from AuxFun import DisplayCompaisonSummary, DisplayRunSummary, MakeSignal


In [ ]:
# Auxiliary Functions


In [ ]:
# Parameters

# Data
numSamples  = 200
noiseStd    = 0.25

λ = 0.5

# Solver
μ               = 0.0025
ρ               = 2
numIterations   = 10_000

# # Verification
ε = 1e-6 #<! Error threshold

## ADMM for TV Denoising

The _Total Variation Denoising_ is a sparse based module which promotes the model of a _piece wise constant_ signal.  
Its general form is given as:

$$ \arg \min_{\boldsymbol{x}} \frac{1}{2} {\left\| \boldsymbol{A} \boldsymbol{x} - \boldsymbol{b} \right\|}_{2}^{2} + \lambda {\left\| \boldsymbol{D} \boldsymbol{x} \right\|}_{1} $$

Where $\lambda {\left\| \boldsymbol{D} \boldsymbol{x} \right\|}_{1}$ is the regularization term with $\lambda \geq 0$ sets the regularization level.  
The matrix $\boldsymbol{D}$ represent the [Finite Difference operator](https://en.wikipedia.org/wiki/Finite_difference).  
In this case the regularization term promotes sparsity over the 1st derivative which implies piece wise constant signal.  

* <font color='brown'>(**#**)</font> There are variations of the [Finite Difference Coefficients](https://en.wikipedia.org/wiki/Finite_difference_coefficient).
* <font color='brown'>(**#**)</font> The regularization is a non smooth function. It requires _non smooth solver_.

This notebooks covers:
 - The solution of the above problem using the ADMM Method.  
 - Comparing the convergence of the ADMM method to the accelerated Sub Gradient.

### The ADMM Method

The Alternating Direction Method of Multipliers (ADMM) is a powerful optimization technique that solves the composite model.  
The canonical form of the problem is:

$$ \arg \min_{\boldsymbol{x}} f \left( \boldsymbol{x} \right) + \lambda g \left( \boldsymbol{P} \boldsymbol{x} \right) $$  

Its main motivation is when $\lambda g \left( \boldsymbol{P} \boldsymbol{x} \right)$ does not have an efficient Prox Operator while $\lambda g \left( \boldsymbol{x} \right)$ does.  
In order to solve the problem the ADMM method breaks the problem into smaller subproblems:

$$ \arg \min_{\boldsymbol{x}, \boldsymbol{z}} f \left( \boldsymbol{x} \right) + \lambda g \left( \boldsymbol{z} \right), \; \text{ subject to } \; \boldsymbol{P} \boldsymbol{x} = \boldsymbol{z} $$

Each subproblem is simpler and easier to solve while ensuring convergence to a global solution:

 - $\arg \min_{\boldsymbol{x}} f \left( \boldsymbol{x} \right) + \frac{\rho}{2} {\left\| \boldsymbol{P} \boldsymbol{x} - \boldsymbol{z}^{k - 1} + \boldsymbol{w}^{k - 1} \right\|}_{2}^{2}$.
 - $\arg \min_{\boldsymbol{z}} \lambda g \left( \boldsymbol{z} \right) + \frac{\rho}{2} {\left\| \boldsymbol{P} \boldsymbol{x}^{k} - \boldsymbol{z} + \boldsymbol{w}^{k - 1} \right\|}_{2}^{2} = \operatorname{prox}_{\frac{\lambda}{\rho} g} \left( \boldsymbol{P} \boldsymbol{x}^{k} - \boldsymbol{z}^{k} \right)$.
 - $\boldsymbol{w}^{k} = \boldsymbol{w}^{k - 1} + \left( \boldsymbol{P} \boldsymbol{x}^{k} - \boldsymbol{z}^{k} \right)$.

ADMM is ideal for large scale problems or when the objective has separable structure.  
Its convergence is similar to Accelerated Proximal Gradient Descent yet in practice it is faster and more robust.

* <font color='brown'>(**#**)</font> The minimization updates is in [_Gauss Seidel Method_](https://en.wikipedia.org/wiki/Gauss%E2%80%93Seidel_method) style.  
Namely each step uses the latest version of the parameters and not the ones from the previous iteration.
* <font color='brown'>(**#**)</font> There are canonical forms of ADMM. For instance, in case $f$ is the Linear Least Squares problem.  
  The efficiency of the method depends on utilizing those forms properly.
* <font color='brown'>(**#**)</font> The ADMM is a very robust method. It has a single hyper parameter, $\rho$.  
  It will converge given any non negative value of $\rho$, yet it will have effect on the speed. 
* <font color='brown'>(**#**)</font> The parameter $\rho$ balances between feasibility (Higher value) to minimization (Lower value).  
There are adaptive policies for setting ${\rho}_{k}$. See [Distributed Optimization and Statistical Learning via the Alternating Direction Method of Multipliers](https://stanford.edu/~boyd/papers/pdf/admm_distr_stats.pdf) section `3.4.1`.



## Generate Data


The data is a Piece Wise Constant signal to match the model of the problem.

* <font color='brown'>(**#**)</font> The function `MakeSignal`  is based on code by [Ivan W. Selesnick - Total Variation Denoising](https://eeweb.engineering.nyu.edu/iselesni/lecture_notes/TVDmm).

In [ ]:
# Generate / Load the Data

# Model Data
vS = MakeSignal('Blocks', numSamples)
vY = vS + (noiseStd * np.random.randn(numSamples))

mD = sp.sparse.spdiags([-np.ones(numSamples), np.ones(numSamples)], [0, 1], numSamples - 1, numSamples) #<! Different than MATLAB in the length required

mX = np.zeros(shape = (numIterations, numSamples)) #<! Initialization by zeros

dSolverData = {}


In [ ]:
# Display Data 

hF, hA = plt.subplots(figsize = (10, 6))
hA.plot(range(numSamples), vS, lw = 2, label = 'Signal Model')
hA.plot(range(numSamples), vY, ls = 'None', marker = '*', ms = 5, label = 'Signal Samples')
hA.set_title('Signal Model and Signal Samples (Noisy)')
hA.set_xlabel('Sample Index')
hA.set_ylabel('Sample Value')

hA.legend();

* <font color='red'>(**?**)</font> How the gradient of the signal model would look like?
* <font color='red'>(**?**)</font> How the gradient of the signal samples  would look like?

## Total Variation Denoising

This section defines the problem and solve it using the _ADMM_.

### Objective Function

The objective function of the TV Denoising:

$$ \arg \min_{\boldsymbol{x}} \underbrace{\frac{1}{2} {\left\| \boldsymbol{x} - \boldsymbol{y} \right\|}_{2}^{2}}_{\text{Fidelity}} + \underbrace{\lambda {\left\| \boldsymbol{D} \boldsymbol{x} \right\|}_{1}}_{\text{Regularization}} $$

The format of fidelity term and regularization is common in [_Inverse Problem_](https://en.wikipedia.org/wiki/Inverse_problem) which are one of the most challenging types of problems in Engineering.  
The regularization is modelling a desired effect on the output. In our case promoting sparse derivative which implies a _piece wise constant_ signal.

* <font color='brown'>(**#**)</font> For simplicity the explicit sparse matrix is used above. In many cases it is better to use an operator.  
In this case by applying a convolution.

In [ ]:
# Objective Function

#===========================Fill This===========================#
# 1. Implement the objective function. 
#    Given a vector of `vX` it returns the objective.
# 2. The implementation should be using a Lambda Function.
# !! You may `np.square()` and / or `np.linalg.norm()`.

hObjFun = lambda vX: 0.5 * np.square(np.linalg.norm(vX - vY)) + λ * np.linalg.norm(mD @ vX, ord = 1)
#===============================================================#

* <font color='red'>(**?**)</font> How would the least squares (With no regularization, $\lambda = 0$) solution look like?

## Analysis

This section solves the problem in 3 ways:

 - DCP Solver: As the problem is _convex_ and relatively small it can be solved by a DCP solver.
 - Accelerated Sub Gradient: Iterative method using the _sub gradient_ of the regularization term with acceleration.
 - ADMM: Iterative separable method which can utilize the Prox operator.


### DCP Solver

Solving the problem using a DCP Solver.

In [ ]:
# DCP Solution
# The Total Variation Denoising model.
# Solved using `CVXPY`.

startTime = time.time()

solverString = 'CVXPY'

#===========================Fill This===========================#
# 1. Create the auxiliary variable `vX`.
# 1. Define the objective function.
# 3. Define the constraints.
# 4. Solve the problem using `CVXPY`.
# !! You may use list operations to define constraints.

vX = cp.Variable(numSamples) #<! Objective Variable

cpObjFun = cp.Minimize(0.5 * cp.sum_squares(vX - vY) + λ * cp.norm(mD @ vX, 1)) #<! Objective Function
cpConst  = [] #<! Constraints
oCvxPrb  = cp.Problem(cpObjFun, cpConst) #<! Problem
#===============================================================#

oCvxPrb.solve(solver = cp.SCS)

assert (oCvxPrb.status == 'optimal'), 'The problem is not solved.'
print('Problem is solved.')

vX = vX.value

runTime = time.time() - startTime

In [ ]:
# Storing Results

DisplayRunSummary(solverString, hObjFun, vX, runTime, oCvxPrb.status)

dSolverData[solverString] = {'vX': vX, 'objVal': hObjFun(vX)}


### Accelerated Sub Gradient Solver

This section implements the Accelerated Sub Gradient Solver.

In [ ]:
# Gradient Function

#===========================Fill This===========================#
# 1. Implement the gradient function of (1/2) * || x - y ||_2^2 + λ * || D x ||_1. 
#    Given a vector `vX` it returns the gradient at `vX`.
# 2. The implementation should be using a Lambda Function.
# !! You may pre calculate terms for efficient code.

hGradFun = lambda vX: (vX - vY) + (λ * mD.T @ np.sign(mD @ vX))
#===============================================================#

In [ ]:
# Prox Function

hProxFun = lambda vY, λ: vY

In [ ]:
# Sub Gradient Solution
# The Total Variation Denoising model.
# Solved using Accelerates Sub Gradient Method.

hMuK = lambda kk: μ #<! Try using the 1/L 
# hMuK = lambda kk: 1 / math.sqrt(kk) #<! Classic Sub Gradient

startTime = time.time()

solverString = 'Accelerated Sub Gradient'

oSubGrad = ProxGradientDescent(np.zeros(numSamples), hGradFun, μ, λ, hProxFun = hProxFun, useAccel = True)
lX = oSubGrad.ApplyIterations(numIterations, logArg = True)

runTime = time.time() - startTime

mX = np.array(lX)



In [ ]:
# Storing Results

vX = np.copy(mX[-1])

DisplayRunSummary(solverString, hObjFun, vX, runTime)

dSolverData[solverString] = {'vX': vX, 'objVal': hObjFun(vX), 'mX': np.copy(mX)}


### ADMM Solver

This section implements the ADMM.  

> [!TIP]  
> For $k \in \left\{ 1, 2, \ldots, K \right\}$:
>    1. $\boldsymbol{x}^{k} = \arg \min_{\boldsymbol{x}} f \left( \boldsymbol{x} \right) + \frac{\rho}{2} {\left\| \boldsymbol{P} \boldsymbol{x} - \boldsymbol{z}^{k - 1} + \boldsymbol{w}^{k - 1} \right\|}_{2}^{2}$
>    2. $\boldsymbol{z}^{k} = \arg \min_{\boldsymbol{z}} \lambda g \left( \boldsymbol{z} \right) + \frac{\rho}{2} {\left\| \boldsymbol{P} \boldsymbol{x}^{k} - \boldsymbol{z} + \boldsymbol{w}^{k - 1} \right\|}_{2}^{2}$
>    3. $\boldsymbol{w}^{k} = \boldsymbol{w}^{k - 1} + \left( \boldsymbol{P} \boldsymbol{x}^{k} - \boldsymbol{z}^{k} \right)$

Where $\arg \min_{\boldsymbol{z}} \lambda g \left( \boldsymbol{z} \right) + \frac{\rho}{2} {\left\| \boldsymbol{P} \boldsymbol{x}^{k} - \boldsymbol{z} + \boldsymbol{w}^{k - 1} \right\|}_{2}^{2} = \operatorname{prox}_{\frac{\lambda}{\rho} g} \left( \boldsymbol{P} \boldsymbol{x}^{k} - \boldsymbol{z}^{k} \right)$.

In [ ]:
# ADMM Function

#===========================Fill This===========================#
# 1. Implement the ADMM solver. 
#    Given a matrix `mX` with shape `(numIter, dataDim)` it applies the method.
# 2. An input parameter is `hMinFun` which is callable `hMinFun(vZ, vW, ρ)`.  
#    It minimizes the problem with respect to `vX`.
# 3. An input parameter is `hProxFun` which is callable `hProxFun(vY, λ)`.  
#    It minimizes the problem with respect to `vZ`.
# 4. The initial value is given by `mX[0, :]`.
# !! Do not overwrite `mX[0, :]`.
# !! Follow the doc string of the function.

def ADMM( mX: np.ndarray, hMinFun: Callable[[np.ndarray, np.ndarray, float], np.ndarray], hProxFun: Callable[[np.ndarray, float], np.ndarray], mP: np.ndarray, /, *, ρ: float = 1.0, λ: float = 1.0 ) -> np.ndarray:
    """
    Alternating Direction Method of Multipliers (ADMM) for Solving Composite Optimization Problems.

    This class implements the ADMM algorithm to solve problems of the form:

        F(x) = f(x) + λ * g(P * x)    ->    f(x) + λ * g(z), subject to P * x - z = 0

    Where:
    - `f(x)` is a function that is easily minimized (e.g., least squares).
    - `g(z)` is a function that has an efficient proximal mapping.
    - `P` is a linear operator (matrix) that transforms `x` into `z`.

    ADMM splits the problem into subproblems that alternately minimize over `x` and `z`, with a dual update for the constraint violation.  
    This method is well suited for large scale or distributed optimization problems where the objective is separable or involves a linear constraint.

    Parameters:
    ----------
    mX : np.ndarray
        A 2D array of shape `(numIter, dataDim)`, where each row represents a point in the optimization process.
        The initial point is provided in `mX[0]`, and subsequent points are updated in place.

    hMinFun : Callable[[np.ndarray, np.ndarray, float], np.ndarray]
        A function that minimizes the function `f(x)` and a quadratic penalty term `(ρ / 2) * || P * x - z + w ||_2^2`.
        This function should take three arguments: the current values of `z`, the dual variable `w`, and the penalty parameter `ρ`, 
        and return the next `x` value that minimizes `f(x) + (ρ / 2) * || P * x - z + w ||_2^2`.

    hProxFun : Callable[[np.ndarray, float], np.ndarray]
        A function that computes the proximal mapping of `g(z)` with respect to the current value of `P * x + w`.
        This function should take two inputs: the current value of `P * x + w`, and the scaled penalty parameter `λ / ρ`, 
        and return the next `z` value that minimizes `λ * g(z) + (ρ / 2) * || P * x - z + w ||_2^2`.

    mP : np.ndarray
        The linear operator (matrix) that transforms the variable `x` into `z`, i.e., the matrix `P` in the constraint `Px - z = 0`.

    ρ : float, optional
        Penalty parameter for the augmented Lagrangian. This parameter controls the tradeoff between the primal and dual residuals 
        in the optimization. Larger values of `ρ` put more emphasis on satisfying the constraint `P * x - z = 0`.
        Range: (0, inf).
        Default: `1.0`.

    λ : float, optional
        The regularization parameter that weights the term `g(z)` in the objective function. 
        Range: (0, inf).
        Default: `1.0`.

    Returns:
    -------
    mX : np.ndarray
        The updated sequence of points after performing the optimization. The final point can be found
        in `mX[-1]`.
    
    Notes:
    ------
    - ADMM is an efficient and flexible method for solving optimization problems that involve separable objectives and 
      linear constraints.
    - The algorithm alternates between minimizing `x` and `z`, which allows for parallelization or distributed 
      computation in some applications.
    - Convergence is typically achieved when the primal and dual residuals (measures of constraint violation and dual 
      update) become sufficiently small.
    """

    numIter = np.size(mX, 0)
    
    # Initialization
    vZ = np.empty(np.size(mP, 0)) #<! Separable variable
    vW = np.empty(np.size(mP, 0)) #<! Dual variable (Lagrangian Multiplier)
    
    # Steady state
    for kk in range(1, numIter):
        mX[kk] = hMinFun(vZ, vW, ρ) #<! Solves \arg \min_x f(x) + (ρ / 2) * || P * x - z + w ||_2^2
        vZ[:]  = hProxFun(mP @ mX[kk] + vW, λ / ρ) #<! Solves \arg \min_z λ * g(z) + (ρ / 2) * || P * x - z + w ||_2^2
        vW[:] += mP @ mX[kk] - vZ #<! Dual variable update
    
    return mX

#===============================================================#

In [ ]:
# ADMM Auxiliary Functions
# Implement the `hMinFun()` and `hProxFun()`.

# Minimization Function
#===========================Fill This===========================#
# 1. Implement the minimization function `hMinFun()`.
#    The function minimizes \arg \min_x 0.5 * || x - y ||_2^2 + (ρ / 2) * || D * x - z + w ||_2^2. 
#    Given the vectors `vZ` and `vW` and the parameter `ρ` it returns the optimal value for `vX`.
# 2. The implementation should be using a Lambda Function.
# !! You may solve the LS problem or try to vanish the gradient.
# !! You may pre calculate terms for efficient code.
# !! You may assume `ρ` is a known constant to improve performance.

mDD = sp.sparse.eye(numSamples) + ρ * (mD.T @ mD) #<! In practice, much better use operators
# Cholesky from `Cholmod` requires `CSC` matrices
mDC = cholesky(sp.sparse.csc_matrix(mDD)) #<! An alternative: https://gist.github.com/omitakahiro/c49e5168d04438c5b20c921b928f1f5d

hMinFun = lambda vZ, vW, ρ: mDC(vY + ρ * mD.T @ (vZ - vW)) #<! Normal equations (Squared the condition number)
#===============================================================#

# Prox Function
#===========================Fill This===========================#
# 1. Implement the prox operator function of the `λ || z ||_1 + (ρ / 2) * || z - (w + D x) ||_2^2` function. 
#    Given a vector `vY` and `λ` it returns the proximal at `vY`.
# 2. The implementation should be using a Lambda Function.
# !! You may assume `λ` > 0.
# !! In this case `vY = w + D x`. Assume all given is `vY`.

hProxFun = lambda vY, λ: np.maximum(np.fabs(vY) - λ, 0) * np.sign(vY) #<! Prox L1
#===============================================================#



In [ ]:
# ADMM Solution
# The Total Variation Denoising model.
# Solved using ADMM Method.

startTime = time.time()

solverString = 'ADMM'

mX = ADMM(mX, hMinFun, hProxFun, mD, ρ = ρ, λ = λ)

runTime = time.time() - startTime



In [ ]:
# Storing Results

vX = np.copy(mX[-1])

DisplayRunSummary(solverString, hObjFun, vX, runTime)

dSolverData[solverString] = {'vX': vX, 'objVal': hObjFun(vX), 'mX': np.copy(mX)}


### Display Results

In [ ]:
# Display Results

hF = DisplayCompaisonSummary(dSolverData, hObjFun)

* <font color='blue'>(**!**)</font> Replace `hMuK` with the step size of the Sub Gradient Method.  
This is the difference between "practice" and theory.

In [ ]:
# Display Data 

hF, hA = plt.subplots(figsize = (10, 6))
hA.plot(range(numSamples), vS, lw = 2, label = 'Signal Model')
hA.plot(range(numSamples), vY, ls = 'None', marker = '*', ms = 5, label = 'Signal Samples')
hA.plot(range(numSamples), mX[-1], lw = 2, label = 'Denoised Signal')
hA.set_title('Signal Model, Signal Samples (Noisy) and Smoothed Signal')
hA.set_xlabel('Sample Index')
hA.set_ylabel('Sample Value')

hA.legend();

* <font color='red'>(**?**)</font> What the result of $\lambda \to \infty$ would look like?